### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

# Goal-oriented bot in DeepPavlov

The tutor is focused on building a goal-oriented dialogue system:

0. [Data preparation](#0.-Data-Preparation)
1. [Build database of items](#1.-Build-database-of-items)
2. [Build Slot Filler](#2.-Build-Slot-Filler)
3. [Train bot](#3.-Train-bot)

An example of the final model served as a telegram bot is:

![gobot_example.png](img/gobot_example.png)

## 0. Data Preparation

The tutor's dialogue system will be on the domain of restaurant booking. [Dialogue State Tracking Challenge 2 (DSTC-2)](http://camdial.org/~mh521/dstc/) dataset provides dialogues of a human talking to a booking system labelled with slots and dialogue actions. The labels are will be used for training a dialogue policy network.

See below a small chunk of the data. 

In [1]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2019-09-03 17:22:01.769 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my_data/simple-dstc2-trn.json]
2019-09-03 17:22:01.792 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my_data/simple-dstc2-val.json]
2019-09-03 17:22:01.843 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my_data/simple-dstc2-tst.json]
2019-09-03 17:22:01.907 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 922 samples in train split.
2019-09-03 17:22:01.907 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-03 17:22:01.908 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 samples in test split.


In [2]:
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-trn.json  simple-dstc2-val.json
simple-dstc2-trn.full.json  simple-dstc2-tst.json


The training/validation/test data is stored in json files (`simple-dstc2-trn.json`, `simple-dstc2-val.json` and `simple-dstc2-tst.json`):

In [3]:
!head -n 101 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },
    {
      "speaker": 1,
      "text": "any",
      "slots": [
        [
          "this",
          "dontcare"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What part of town do you have in mind?",
      "slots": [],
      "act": "request_area"
    },
    {
      "speaker": 1,
      "text": "south",
      "slots": [
        [
          "area",
          "south"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "api_call area=\"south\" food

In [4]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

In [4]:
from deeppavlov.dataset_iterators.dstc2_ner_iterator import Dstc2NerDatasetIterator

iterator = Dstc2NerDatasetIterator(data, slot_values_path='my_bot/slotfill/dstc_slot_vals.json')


utterance = {'text': 'cheap restaurant', 'slots': [['pricerange', 'cheap']]}
text      = cheap restaurant
slots     = [('pricerange', 'cheap')]


utterance = {'text': 'any', 'slots': [['this', 'dontcare']]}
text      = any
slots     = [('this', 'dontcare')]


utterance = {'text': 'south', 'slots': [['area', 'south']]}
text      = south
slots     = [('area', 'south')]


utterance = {'text': 'api_call area="south" food="dontcare" pricerange="cheap"', 'act': 'api_call', 'slots': [['area', 'south'], ['pricerange', 'cheap'], ['food', 'dontcare']]}
text      = api_call area="south" food="dontcare" pricerange="cheap"
slots     = [('area', 'south'), ('pricerange', 'cheap'), ('food', 'dontcare')]


utterance = {'text': 'south', 'db_result': {'food': 'chinese', 'pricerange': 'cheap', 'area': 'south', 'addr': 'cambridge leisure park clifton way cherry hinton', 'phone': '01223 244277', 'postcode': 'c.b 1, 7 d.y', 'name': 'the lucky star'}, 'slots': [['area', 'south']]}
text      = south
slots    



utterance = {'text': 'i dont care', 'slots': [['this', 'dontcare']]}
text      = i dont care
slots     = [('this', 'dontcare')]


utterance = {'text': 'api_call area="west" food="italian" pricerange="dontcare"', 'act': 'api_call', 'slots': [['area', 'west'], ['food', 'italian'], ['pricerange', 'dontcare']]}
text      = api_call area="west" food="italian" pricerange="dontcare"
slots     = [('area', 'west'), ('food', 'italian'), ('pricerange', 'dontcare')]


utterance = {'text': 'i dont care', 'db_result': {'pricerange': 'moderate', 'area': 'west', 'addr': '21 - 24 northampton road', 'phone': '01799 521260', 'food': 'italian', 'postcode': 'c.b 3, 0 a.d', 'name': 'prezzo'}, 'slots': [['this', 'dontcare']]}
text      = i dont care
slots     = [('this', 'dontcare')]


utterance = {'text': 'Prezzo is a nice place in the west of town serving tasty italian food.', 'act': 'inform_area+inform_food+offer_name', 'slots': [['area', 'west'], ['name', 'prezzo'], ['food', 'italian']]}
text      = Pr



utterance = {'text': 'type of food', 'slots': [['slot', 'food']]}
text      = type of food
slots     = []


utterance = {'text': 'Kohinoor is a great restaurant.', 'act': 'offer_name', 'slots': [['name', 'kohinoor']]}
text      = Kohinoor is a great restaurant.
slots     = [('name', 'kohinoor')]


utterance = {'text': 'type of food', 'slots': [['slot', 'food']]}
text      = type of food
slots     = []


utterance = {'text': 'Kohinoor serves indian food.', 'act': 'inform_food+offer_name', 'slots': [['food', 'indian'], ['name', 'kohinoor']]}
text      = Kohinoor serves indian food.
slots     = [('food', 'indian'), ('name', 'kohinoor')]


utterance = {'text': 'im looking for a cheap restaurant that serves', 'slots': [['pricerange', 'cheap']]}
text      = im looking for a cheap restaurant that serves
slots     = [('pricerange', 'cheap')]


utterance = {'text': 'Sorry would you like chinese or russian food?', 'act': 'select_food', 'slots': [['food', 'russian'], ['food', 'chinese']]}
text 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



You can now iterate over batches of preprocessed DSTC-2 dialogs:

In [5]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

User utterances:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {   'prev_resp_act': 'welcomemsg',
        'slots': [['pricerange', 'moderate']],
        'text': 'i would like a moderately priced restaurant'},
    {   'prev_resp_act': 'request_food',
        'slots': [['food', 'spanish']],
        'text': 'spanish food'},
    {   'db_result': {   'addr': '14 -16 bridge street',
                         'area': 'centre',
                         'food': 'spanish',
                         'name': 'la tasca',
                         'phone': '01223 464630',
                         'postcode': 'c.b 2, 1 u.f',
                         'pricerange': 'moderate'},
        'prev_resp_act': 'api_call',
        'slots': [['food', 'spanish']],
        'text': 'spanish food'},
    {   'prev_resp_act': 'inform_food+inform_pricerange+offer_name',
        'slots': [['slot', 'addr']],
        'text': 'what is the address'},
    {   'prev_resp_act': 'inform_addr+offer_name',
        '

In [6]:
!cp my_data/simple-dstc2-trn.json my_data/simple-dstc2-trn.full.json

In [7]:
import json

NUM_TRAIN = 100


with open('my_data/simple-dstc2-trn.full.json', 'rt') as fin,\
    open('my_data/simple-dstc2-trn.json', 'wt') as fout:
        data = json.load(fin)
        
        print(f"Train set contains {NUM_TRAIN}/{len(data)} dialogues.")
        json.dump(data[:NUM_TRAIN], fout, indent=2)


Train set contains 100/100 dialogues.


## 1. Build database of items

&nbsp;
![gobot_database.png](img/gobot_database.png)
&nbsp;

For a valid goal-oriented bot there should be a `database` of relevant items. In the case of restaurant booking it will contain all available restaurants and their info.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]
           
The dialogues in the training dataset should contain a `"db_result"` dictionary key. It is required for turns where system performs a special type of external action: an api call to the database of items. `"db_result"` should contain the result of the api call:

In [8]:
!head -n 78 my_data/simple-dstc2-trn.json | tail +51

    {
      "speaker": 2,
      "text": "api_call area=\"south\" food=\"dontcare\" pricerange=\"cheap\"",
      "db_result": {
        "food": "chinese",
        "pricerange": "cheap",
        "area": "south",
        "addr": "cambridge leisure park clifton way cherry hinton",
        "phone": "01223 244277",
        "postcode": "c.b 1, 7 d.y",
        "name": "the lucky star"
      },
      "slots": [
        [
          "area",
          "south"
        ],
        [
          "pricerange",
          "cheap"
        ],
        [
          "food",
          "dontcare"
        ]
      ],
      "act": "api_call"
    },


In [9]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"],
                           save_path="my_bot/db.sqlite")

2019-09-03 17:22:12.118 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-03 17:22:12.119 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


Set `primary_keys` to a list of slot names that have unique values for different items (common SQL term). For the case of DSTC-2, the primary slot is restaurant name.

Let's find all `"db_result"` api call results and add it to our database of restaurants:

In [10]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

Adding 1843 items.


##### Interacting with database

We can now play with the database and make requests to it:

In [11]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'area': 'south',
   'addr': 'cambridge leisure park clifton way cherry hinton',
   'postcode': 'c.b 1, 7 d.y',
   'phone': '01223 244277',
   'name': 'the lucky star',
   'food': 'chinese',
   'pricerange': 'cheap'},
  {'area': 'south',
   'addr': 'cambridge leisure park clifton way',
   'postcode': 'c.b 1, 7 d.y',
   'phone': '01223 327908',
   'name': 'nandos',
   'food': 'portuguese',
   'pricerange': 'cheap'}]]

In [12]:
!ls my_bot

db.sqlite  slotfill


## 2. Build Slot Filler

&nbsp;
![gobot_slotfiller.png](img/gobot_slotfiller.png)
&nbsp;

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of each slot
 
The data should be in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

Let's use a simple non-trainable slot filler that relies on levenshtein distance:

In [13]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-09-03 17:22:18.612 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 11.3MB/s]
2019-09-03 17:22:18.617 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [14]:
!ls my_bot/slotfill

dstc_slot_vals.json


In [15]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


##### Metric scores on valid&test

Let's check performance of our slot filler on DSTC-2 dataset:

In [16]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

slotfill_config = read_json(configs.ner.slotfill_simple_dstc2_raw)

We take [original DSTC2 slot-filling config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/ner/slotfill_dstc2_raw.json) and change variables determining data paths:

In [17]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [18]:
from deeppavlov import evaluate_model

slotfill = evaluate_model(slotfill_config);

2019-09-03 17:22:30.687 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-03 17:22:30.694 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-03 17:22:30.739 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-03 17:22:30.814 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 922 samples in train split.
2019-09-03 17:22:30.814 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-03 17:22:30.815 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

{"valid": {"eval_examples_count": 1253, "metrics": {"slots_accuracy": 0.933}, "time_spent": "0:00:25"}}
{"test": {"eval_examples_count": 1190, "metrics": {"slots_accuracy": 0.9487}, "time_spent": "0:00:23"}}


We've got slot accuracy of **93% on valid** set and **94% on test** set.

##### Interacting with slot filler

In [19]:
from deeppavlov import build_model

slotfill = build_model(slotfill_config)

In [20]:
slotfill(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

##### Dumping slot filler's config

Saving slotfill config file to disk (we will require it's path later):

In [21]:
import json

json.dump(slotfill_config, open('my_bot/slotfill_config.json', 'wt'))

In [22]:
!ls my_bot

db.sqlite  slotfill  slotfill_config.json


## 3. Train bot

Let's assemble all modules together and train the final module: dialogue policy network.

&nbsp;
![gobot_policy.png](img/gobot_policy.png)
&nbsp;

Policy network decides which action the system should take on each turn of a dialogue: should it say goodbye, request user's location or make api call to a database.

The policy network is a recurrent neural network (recurrent over utterances represented as bags of words) and a dense layer with softmax function on top. The network classifies user utterance into one of predefined system actions.

&nbsp;
![gobot_templates.png](img/gobot_templates.png)
&nbsp;

All actions available for the system should be listed in a `simple-dstc2-templates.txt` file. Each action should be associated with a string of the corresponding system response.

Templates should be in the format `<act>TAB<template>`, where `<act>` is a dialogue action and `<template>` is the corresponding response. Response text might contain slot type names, where every `#slot_type` will be filled with the slot value from a dialogue state.

In [23]:
!head -n 10 my_data/simple-dstc2-templates.txt

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.
canthelp_area_food_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #food food.
canthelp_area_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #area american food.
canthelp_food	I am sorry but there is no #food restaurant that matches your request.
canthelp_food_pricerange	Sorry there is no #food restaurant in the #pricerange price range.
confirm-domain	You are looking for a restaurant is that right?


So, actions are actually classes we classify over. And `simple-dstc2-templates.txt` contains the set of classes.

To train the dialogue policy network for classification task you need action label for each system utterance in training dialogues. The DSTC-2 contains `"act"` dictionary key that contains action associated with current response.

The cell below provides an example of training data for the policy network.

In [24]:
!head -n 24 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },


Let's **construct the final pipeline** of a dialogue system.

We take [default DSTC2 bot config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_dstc2.json) ([more configs](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot) are available) and change sections responsible for 
- templates,
- database,
- slot filler,
- embeddings,
- data and model load/save paths.

In [25]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_simple_dstc2)

**Configure** bot to use **templates**:

In [26]:
gobot_config['chainer']['pipe'][-1]['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['template_path'] = 'my_data/simple-dstc2-templates.txt'

**Configure** bot to use our built **database**:

In [27]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["name"],
    'save_path': 'my_bot/db.sqlite'
}

**Configure** bot to use levenshtein distance based **slot filler**:

In [28]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_config.json'

gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['pricerange', 'this', 'area', 'food']

You can use a simple **bag-of-words as embedder** (by default):

In [29]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Specify train/valid/test **data path** and **path to save** the final bot model:

In [30]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

The whole dialogue system pipeline looks like this:
    
![gobot_pipeline.png](img/gobot_pipeline.png)

##### Training policy network

In [31]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 8 # set batch size
gobot_config['train']['max_batches'] = 200 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 50 # evaluate on full 'valid' split each 50 batches
gobot_config['train']['log_every_n_batches'] = 50 # evaluate on 30 batches of 'train' split every 50 batches
gobot_config['train']['log_on_k_batches'] = 50

train_model(gobot_config);

2019-09-03 17:24:13.826 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-03 17:24:13.832 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-03 17:24:13.932 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-03 17:24:13.968 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 922 samples in train split.
2019-09-03 17:24:13.969 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-03 17:24:13.969 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.
2019-09-03 17:24:15.157 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-templates.txt]
2019-09-03 17:24:15.158 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 46 templates loaded.
2019-09-03 17:24:15.159 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 249


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use standard file APIs to check for files with this prefix.


2019-09-03 17:24:15.728 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 251: [initializing `GoalOrientedBot` from scratch]
2019-09-03 17:25:29.833 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.014
2019-09-03 17:25:29.833 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
2019-09-03 17:25:29.834 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-09-03 17:25:29.903 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 764: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]


...

INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model
{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4189}, "time_spent": "0:01:15"}}


2019-09-03 17:36:37.648 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-03 17:36:37.649 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-03 17:36:37.650 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-03 17:36:37.735 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-templates.txt]
2019-09-03 17:36:37.735 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 46 templates loaded.
2019-09-03 17:36:37.736 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 24

{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4115}, "time_spent": "0:01:12"}}


2019-09-03 17:36:38.292 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
2019-09-03 17:36:38.293 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 770: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-09-03 17:36:38.297 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model


Training on 1k dialogues takes from 10 to 30 minutes depending on gpu/cpu.

See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of the training process.

##### Metric scores on valid&test

Calculating **accuracy** of trained bot: whether predicted system responses match true responses (full string match).

In [32]:
import evaluate_model

evaluate_model(gobot_config);

2019-09-03 17:36:38.329 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-03 17:36:38.337 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-03 17:36:38.376 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-03 17:36:38.524 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 922 samples in train split.
2019-09-03 17:36:38.524 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-03 17:36:38.524 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model
{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4189}, "time_spent": "0:01:16"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4115}, "time_spent": "0:01:12"}}


With settings of `max_batches=200`, valid accuracy `~ 0.4` and test accuracy is `0.4`.

##### Chatting with bot

In [34]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-09-03 17:40:42.935 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-03 17:40:42.936 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-03 17:40:42.937 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-03 17:40:43.21 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-templates.txt]
2019-09-03 17:40:43.22 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 46 templates loaded.
2019-09-03 17:40:43.22 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 249
2

INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model


In [35]:
bot(['hi, want chinee food, can you suggest a place to go?'])

2019-09-03 17:40:44.20 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'food': 'chinese'}, got 16 results.


['The lucky star serves chinese food.']

In [36]:
bot(['ok, i want ther address'])

['Sure, the lucky star is on cambridge leisure park clifton way cherry hinton.']

In [37]:
bot(['now i want french food in the center of town'])

2019-09-03 17:40:44.954 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.


['Cote serves french food.']

In [38]:
bot(['thanks, give me their address too'])

['Sure, cote is on bridge street city centre.']

In [39]:
bot(['and post code'])

['The post code of cote is c.b 2, 1 u.f.']

In [40]:
bot(['bye'])

['Can I help you with anything else?']

In [41]:
bot(['no, thanks!'])

['Can I help you with anything else?']

In [42]:
bot.reset()

In [43]:
bot(['hi, is there any cheap restaurant?'])

2019-09-03 17:41:05.363 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'pricerange': 'cheap'}, got 22 results.


['The lucky star is a nice place in the south of town and the prices are cheap.']